# Figure 3: NIRCam Grism Dispersion Lengths for Module A

***
### Table of Contents

1. [Information](#Information)
2. [Imports](#Imports)
3. [Data](#Data)
4. [Generate the Grism Dispersion Lengths Plot](#Generate-the-Grism-Dispersion-Lengths-Plot)
5. [Issues](#Issues)
6. [About this Notebook](#About-this-Notebook)
***

## Information

#### JDox links: 
* [NIRCam Grisms](https://jwst-docs.stsci.edu/display/JTI/NIRCam+Grisms#NIRCamGrisms-Dispersion)
    * Figure 3. NIRCam grism dispersion lengths for module A

## Imports

In [ ]:
import os
import numpy as np
import pandas as pd
from astropy.io import ascii, fits
from astropy.table import Table
from scipy.optimize import fmin
from scipy.interpolate import interp1d
import requests
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline

## Data

#### Data Location: 

The data is stored in a NIRCam JDox Box folder here:
[ST-INS-NIRCAM -> JDox -> nircam_grisms](https://stsci.box.com/s/wu9mo54vi957x50rdirlcg9zkkr3xiaw)

In [ ]:
files = [('https://stsci.box.com/shared/static/i0a9dkp02nnuw6w0xcfd7b42ctxfb8es.fits', 'NIRCam.F250M.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/vfnyk9veote92dz1edpbu83un5n20rsw.fits', 'NIRCam.F250M.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/ssvltwzt7f4y5lfvch2o1prdk5hb2gz2.fits', 'NIRCam.F250M.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/56wjvzx1jf2i5yg7l1gg77vtvi01ec5p.fits', 'NIRCam.F250M.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/v1621dcm44be21n381mbgd2hzxxqrb2e.fits', 'NIRCam.F277W.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/8slec91wj6ety6d8qvest09msklpypi8.fits', 'NIRCam.F277W.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/r42hdv64x6skqqszv24qkxohiijitqcf.fits', 'NIRCam.F277W.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/3vye6ni05i3kdqyd5vs1jk2q59yyms2e.fits', 'NIRCam.F277W.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/twcxbe6lxrjckqph980viiijv8fpmm8b.fits', 'NIRCam.F300M.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/bpvluysg3zsl3q4b4l5rj5nue84ydjem.fits', 'NIRCam.F300M.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/15x7rbwngsxiubbexy7zcezxqm3ndq54.fits', 'NIRCam.F300M.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/a7tqdp0feqcttw3d9vaioy7syzfsftz6.fits', 'NIRCam.F300M.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/i76sb53pthieh4kn62fpxhcxn8lreffj.fits', 'NIRCam.F322W2.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/wgbyfi3ofs7i19b7zsf2iceupzkbkokq.fits', 'NIRCam.F322W2.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/jhk3deym5wbc68djtcahy3otk2xfjdb5.fits', 'NIRCam.F322W2.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/zu3xqnicbyfjn54yb4kgzvnglanf13ak.fits', 'NIRCam.F322W2.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/e2srtf52wnh6vvxsy2aiknbcr8kx2xr5.fits', 'NIRCam.F335M.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/bav3tswdd7lemsyd53bnpj4b6yke5bgd.fits', 'NIRCam.F335M.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/81wm768mjemzj84w1ogzqddgmrk3exvt.fits', 'NIRCam.F335M.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/fhopmyongqifibdtwt3qr682lwdjaf7a.fits', 'NIRCam.F335M.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/j9gd8bclethgex40o7qi1e79hgj2hsyt.fits', 'NIRCam.F356W.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/s23novi3p6qwm9f9hj9wutgju08be776.fits', 'NIRCam.F356W.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/41fnmswn1ttnwts6jj5fu73m4hs6icxd.fits', 'NIRCam.F356W.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/wx3rvjt0mvf0hnhv4wvqcmxu61gamwmm.fits', 'NIRCam.F356W.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/e0p6vkiow4jlp49deqkji9kekzdt4oon.fits', 'NIRCam.F360M.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/xbh0rjjvxn0x22k9ktiyikol7c4ep6ka.fits', 'NIRCam.F360M.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/e7artuotyv8l9wfoa3rk1k00o5mv8so8.fits', 'NIRCam.F360M.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/9r5bmick13ti22l6hcsw0uod75vqartw.fits', 'NIRCam.F360M.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/tqd1uqsf8nj12he5qa3hna0zodnlzfea.fits', 'NIRCam.F410M.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/4szffesvswh0h8fjym5m5ht37sj0jzrl.fits', 'NIRCam.F410M.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/iur0tpbts23lc5rn5n0tplzndlkoudel.fits', 'NIRCam.F410M.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/rvz8iznsnl0bsjrqiw7rv74jj24b0otb.fits', 'NIRCam.F410M.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/sv3g82qbb4u2umksgu5zdl7rp569sdi7.fits', 'NIRCam.F430M.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/mmqv1pkuzpj6abtufxxfo960z2v1oygc.fits', 'NIRCam.F430M.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/84q83haic2h6eq5c6p2frkybz551hp8d.fits', 'NIRCam.F430M.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/3osceplhq6kmvmm2a72jsgrg6z1ggw1p.fits', 'NIRCam.F430M.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/kitx7gdo5kool6jus2g19vdy7q7hmxck.fits', 'NIRCam.F444W.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/ug7y93v0en9c84hfp6d3vtjogmjou9u3.fits', 'NIRCam.F444W.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/0p9h9ofayq8q6dbfsccf3tn5lvxxod9i.fits', 'NIRCam.F444W.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/34hbqzibt5h72hm0rj9wylttj7m9wd19.fits', 'NIRCam.F444W.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/vj0rkyebg0afny1khdyiho4mktmtsi1q.fits', 'NIRCam.F460M.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/ky1z1dpewsjqab1o9hstihrec7h52oq4.fits', 'NIRCam.F460M.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/s93cwpcvnxfjwqbulnkh9ts9ln0fu9cz.fits', 'NIRCam.F460M.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/1178in8zg462es1fkl0mgcbpgp6kgb6t.fits', 'NIRCam.F460M.R.B.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/b855uj293klac8hnoqhrnv8ei0rcvudj.fits', 'NIRCam.F480M.R.A.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/werzjlp3ybxk2ovg6u689zsfpts2t8w3.fits', 'NIRCam.F480M.R.A.2nd.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/yrh5mylru1upbo5rifbz77acn8k1ud6i.fits', 'NIRCam.F480M.R.B.1st.sensitivity.fits'),
         ('https://stsci.box.com/shared/static/oxu6jsg9cn9yqkh3nh646fx0flhw8rej.fits', 'NIRCam.F480M.R.B.2nd.sensitivity.fits')]
         

In [ ]:
def download_file(url, file_name, output_directory='./', overwrite=False):
    """Download a file from Box given the direct URL

    Parameters
    ----------
    url : str
        URL to the file to be downloaded
        
    file_name : str
        The name of the file being downloaded
        
    output_directory : str
        Directory to download file_name into
        
    overwrite : str
        If False and the file to download already exists, the download
        will be skipped. If True, the file will be downloaded regardless
        of whether it already exists in output_directory

    Returns
    -------
    download_filename : str
        Name of the downloaded file
    """
    download_filename = os.path.join(output_directory, file_name)
    if not os.path.isfile(download_filename) or overwrite is True:
        print("Downloading {}".format(file_name))
        with requests.get(url, stream=True) as response:
            if response.status_code != 200:
                raise RuntimeError("Wrong URL - {}".format(url))
            with open(download_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=2048):
                    if chunk:
                        f.write(chunk)
    else:
        print("{} already exists. Skipping download.".format(download_filename))
    return download_filename

#### Load the data

(The next cell assumes you downloaded the data into your ```Users/$(logname)/``` home directory)

In [ ]:
if os.environ.get('LOGNAME') is None:
    raise ValueError("WARNING: LOGNAME environment variable not set!")

In [ ]:
box_directory = os.path.join("/Users/", os.environ['LOGNAME'], "box_data")   
box_directory

In [ ]:
if not os.path.isdir(box_directory):
    try:
        os.mkdir(box_directory)
    except:
        raise OSError("Unable to create {}".format(box_directory))

In [ ]:
for file_info in files:
    file_url, filename = file_info
    outfile = download_file(file_url, filename, output_directory=box_directory)

In [ ]:
grism = "R"
mod = "A"
thr = 0.05 # 5% of peak boundaries
filters = ["F250M","F277W","F300M","F322W2","F335M","F356W","F360M","F410M","F430M","F444W","F460M","F480M"]

In [ ]:
filenames = []
for fil in filters:
    filenames.append(os.path.join(box_directory, "NIRCam.%s.%s.%s.2nd.sensitivity.fits" % (fil,grism,mod)))                 

In [ ]:
filenames

### Define some convenience functions

In [ ]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]


def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]


def find_mid(w,s,w0,thr=0.05):
    fct = interp1d(w,s,bounds_error=None,fill_value='extrapolate')
    def func(x):
        #print "x:",x
        return np.abs(fct(x)-thr)
    
    res = fmin(func,w0)
    
    return res[0]

In [ ]:
tab = []
for fname in filenames:
    d = fits.open(fname)
    w = d[1].data["WAVELENGTH"]
    s = d[1].data["SENSITIVITY"]
    s = s/np.max(s)
    wmin,wmax = np.min(w),np.max(w)
    vg = w<(wmax+wmin)/2.
    w1 = find_mid(w[vg],s[vg],wmin,thr)
    
    vg = w>(wmax+wmin)/2.
    w2 =  find_mid(w[vg],s[vg],wmax,thr)
#     print('\n')
#     print('Filter,lamda1,lamda2,dispersedpix: '+str([fil,w1,w2,(w2-w1)*1000]))
#     print('\n\n')
    
    tab.append([fil,w1,w2,(w2-w1)*1000])
    
columns=['filter','lambda1','lambda2','npix']
df = pd.DataFrame(tab, columns=columns)
table = Table.from_pandas(df)

In [ ]:
tab2 = []
for fname in filenames:
    d = fits.open(fname)
    w = d[1].data["WAVELENGTH"]
    s = d[1].data["SENSITIVITY"]
    s = s/np.max(s)
    wmin,wmax = np.min(w),np.max(w)
    vg = w<(wmax+wmin)/2.
    w1 = find_mid(w[vg],s[vg],wmin,thr)
    
    vg = w>(wmax+wmin)/2.
    w2 =  find_mid(w[vg],s[vg],wmax,thr)
#     print('\n')
#     print('Filter,lamda1,lamda2,dispersedpix: '+str([fil,w1,w2,(w2-w1)*1000]))
#     print('\n\n')
    
    tab2.append([fil,w1,w2,(w2-w1)*1000])
    
columns=['filter','lambda1','lambda2','npix']
df = pd.DataFrame(tab2, columns=columns)
table2 = Table.from_pandas(df)

## Generate the Grism Dispersion Lengths Plot

In [ ]:
f, ax1 = plt.subplots(1,figsize=(15, 20))
ms = 14
for i in np.arange(0,len(table)):
    ax1.plot([table[i]['lambda1'],table[i]['lambda2']], [int(i),int(i)], 'k', lw=3) 
    ax1.plot([table2[i]['lambda1']*2,table2[i]['lambda2']*2], [int(i),int(i)], 'k', alpha=0.3, lw=3) 
    ax1.text(1.5, int(i), table[i]['filter'], va='center', fontsize=22)
    ax1.plot(3.9, int(i), 'kx', ms=ms)

ax1.plot(3.9, 4, 'kx', ms=ms)
ax1.plot(3.9, 3, 'kx', ms=ms)
ax1.plot(3.9, 2, 'kx', ms=ms)
ax1.plot(3.9, 1, 'kx', ms=ms)
ax1.plot(3.9, 0, 'kx', ms=ms)

ax1.text(4.66, 5., 'detector', va='center', fontsize=22,color='red')
ax1.text(4.5, 5.35, '(2040 pixels)', va='center', fontsize=22,color='red')
ax1.text(3.0, -0.8, '-x (Row)', va='center', fontsize=22,color='red')
ax1.text(3.4, -1.5, 'source position', va='center', fontsize=22,color='black')

xticks = ax1.xaxis.get_major_ticks()
xticks[0].label1.set_visible(False)

ax1.set_xlim(1.25, 5.5)

ax1.xaxis.tick_top()
ax1.xaxis.set_label_position('top') 
ax1.xaxis.set_ticks_position('both')

ax1.tick_params(labelsize=20)
plt.setp(ax1.get_yticklines(),visible=False)
ax1.tick_params('x', length=10, width=0.5,direction='in',which='major')

ax1.add_patch(patches.Rectangle((2.2, 3.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, 2.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, 1.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, 0.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, -0.25),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((2.2, 4.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 5.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 6.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 7.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 8.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 9.75),2, 0.5,fill=False,color='red',lw=4))
ax1.add_patch(patches.Rectangle((3.3, 10.75),2, 0.5,fill=False,color='red',lw=4))

ax1.arrow(3.0, -0.45, 0.44, 0, head_width=0.22, head_length=0.1, fc='r', ec='r')
ax1.arrow(3.9, -1.15, 0, 0.6,head_width=0.1,head_length=0.2, fc='k', ec='k')

ax1.annotate('$1^{st}$ order',
            xy=(1.44,-2.0), xycoords='data',
            xytext=(80, 0), textcoords='offset points',
            arrowprops=dict(arrowstyle='-', ls='solid',facecolor='black',lw=4,shrinkB=3),
            horizontalalignment='center', verticalalignment='center',fontsize=18)    
ax1.annotate('$2^{nd}$ order',
            xy=(1.44,-1.6), xycoords='data',
            xytext=(81, 0), textcoords='offset points',
            arrowprops=dict(arrowstyle='-', ls='solid',facecolor='black',alpha=0.3,lw=4,shrinkB=3),
            horizontalalignment='center', verticalalignment='center',fontsize=18) 

ax1.add_patch(patches.Rectangle((1.33, -2.3),0.9, 1,fill=False,color='grey',lw=1))

ax1.set_yticklabels([])
plt.gca().invert_yaxis()
ax1.set_ylim(11.7, -2.5)

f.text(0.5, 0.925,'First-Order Wavelength (microns)', ha='center', fontsize=20)

ax1.xaxis.get_major_ticks()[1].draw = lambda *args:None
ax1.xaxis.get_major_ticks()[-1].draw = lambda *args:None

## Issues

* None

## About this Notebook
**Authors:**   
Dan Coe & Alicia Canipe

**Updated On:**   
April 10, 2019